# The Clustering Methods
1. K-Means
2. Fuzzy C-means or EM
4. DBScan
5. Spectral

## Import the Data

In [127]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn import cluster

from google.colab import drive
drive.mount('/content/gdrive')

%cd "/content/gdrive/Shareddrives/CSCI 5523 - Data Mining Final Project"
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/Shareddrives/CSCI 5523 - Data Mining Final Project
 Balance_deprecated.ipynb   classification.ipynb   data
'Bot Detection.gslides'     cluster_2.ipynb	   data.ipynb
 botDetection.mp4	    cluster.ipynb	   Proposal.gdoc


In [128]:
 # Dataset 1
unbal_train = pd.read_json('data/unbal_train.json').replace({'human': 0, 'bot':1})
unbal_test = pd.read_json('data/unbal_test.json').replace({'human': 0, 'bot':1})
# Dataset 2
bal_train = pd.read_json('data/bal_train.json').replace({'human': 0, 'bot':1})
bal_test = pd.read_json('data/bal_test.json').replace({'human': 0, 'bot':1})
# Dataset 3
crop_unbal_train = pd.read_json('data/crop_unbal_train.json').replace({'human': 0, 'bot':1})
crop_unbal_test = pd.read_json('data/crop_unbal_test.json').replace({'human': 0, 'bot':1})
# Dataset 4
human_train = pd.read_json('data/more_human_train.json').replace({'human': 0, 'bot':1})
human_test = pd.read_json('data/more_human_test.json').replace({'human': 0, 'bot':1})

train_df = [unbal_train, bal_train, crop_unbal_train, human_train]
test_df = [unbal_test, bal_test, crop_unbal_test, human_test]


In [129]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def print_analysis(test, predictions, label):
  print("-----------------------------------")
  print('{} Accuracy score: {}'.format(label, accuracy_score(test, predictions)))
  print('{} Precision score: {}'.format(label, precision_score(test, predictions)))
  print('{} Recall score: {}'.format(label, recall_score(test, predictions)))
  print('{} F1 score: {}'.format(label, f1_score(test, predictions)))
  print("-----------------------------------\n")

In [130]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# # UNBALANCED
# x_train_u = train_u.to_numpy()[:,:-1].astype(int)   # all cols of train_u except for class label
# x_train_u_scaled = scaler.fit_transform(x_train_u)

# y_train_u = train_u.to_numpy()[:,-1].astype(int)    # only class label (last) column of train_u

# x_test_u = test_u.to_numpy()[:,:-1].astype(int)   # all cols of test_u except for class label
# x_test_u_scaled = scaler.transform(x_test_u)

# y_test_u = test_u.to_numpy()[:,-1].astype(int)    # only class label (last) column of test_u

# # BALANCED
# x_train_b = train_b.to_numpy()[:,:-1].astype(int)   # all cols of train_b except for class label
# x_train_b_scaled = scaler.fit_transform(x_train_b)

# y_train_b = train_b.to_numpy()[:,-1].astype(int)    # only class label (last) column of train_b

# x_test_b = test_b.to_numpy()[:,:-1].astype(int)   # all cols of test_b except for class label
# x_test_b_scaled = scaler.transform(x_test_b)

# y_test_b = test_b.to_numpy()[:,-1].astype(int)    # only class label (last) column of test_b

train_data = []
train_labels = []
test_data = []
test_labels = []
for i in range(len(train_df)):
  train = train_df[i]
  test = test_df[i]
  x_train = train.to_numpy()[:, :-1].astype(int)
  x_train_scaled = scaler.fit_transform(x_train)
  train_data.append(x_train_scaled)
  y_train = train.to_numpy()[:, -1].astype(int)
  train_labels.append(y_train)

  x_test = test.to_numpy()[:, :-1].astype(int)
  x_test_scaled = scaler.transform(x_test)
  test_data.append(x_test_scaled)
  y_test = test.to_numpy()[:, -1].astype(int)
  test_labels.append(y_test)

# K-Means Clustering

### K-means training and resulting centroids

In [131]:
k_means = []
for i in range(len(train_data)):
  print('Dataset '+ str(i+1))
  data = pd.DataFrame(train_data[i], columns = train_df[i].columns[:-1])
  k = cluster.KMeans(n_clusters=2, max_iter=100, random_state=1)
  k.fit(data)
  k_means.append(k)
  centroids = k.cluster_centers_
  print(pd.DataFrame(centroids, columns=data.columns))

# # K-means on Unbalanced Data
# data = pd.DataFrame(x_train_u_scaled, columns=train_u.columns[:-1])
# k_means = cluster.KMeans(n_clusters=2, max_iter=100, random_state=1)
# k_means.fit(data) 
# labels = k_means.labels_
# pd.DataFrame(labels, columns=['Cluster ID'])

Dataset 1
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.089679    -0.237592  ...         -0.147124       -0.158361
1  0.634291     1.680458  ...          1.040588        1.120071

[2 rows x 9 columns]
Dataset 2
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.157993    -0.423617  ...         -0.236015       -0.253172
1  0.344719     0.924274  ...          0.514952        0.552387

[2 rows x 9 columns]
Dataset 3
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.093626    -0.240170  ...         -0.148566       -0.149068
1  0.646861     1.659331  ...          1.026442        1.029911

[2 rows x 9 columns]
Dataset 4
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.000607    -0.000138  ...          0.000056        0.000003
1  4.735021     1.079716  ...         -0.435631       -0.024269

[2 rows x 9 columns]


### Evalute Models on Training Data

In [132]:
for i in range(len(k_means)):
  print('Dataset '+ str(i+1))
  k = k_means[i]
  labels = np.uint(k.predict(train_data[i]))
  # flip the labels since the random assignment got them backwards
  labels_flipped = np.logical_not(labels).astype(int)
  print_analysis(labels, train_labels[i], 'Training')
  print_analysis(labels_flipped, train_labels[i], 'Training Flipped')


# # Evaluate on training data
# labels = k_means.predict(x_train_u_scaled)
# labels = labels.reshape(-1, 1)
# cols = train_u.columns.tolist()
# cols.append('Cluster ID')
# check = pd.DataFrame(np.concatenate((train_u, labels), axis=1), columns=cols)
# check

Dataset 1
-----------------------------------
Training Accuracy score: 0.08318286378589626
Training Precision score: 0.027502652989034312
Training Recall score: 0.18630191693290735
Training F1 score: 0.04792972362067194
-----------------------------------

-----------------------------------
Training Flipped Accuracy score: 0.9168171362141038
Training Flipped Precision score: 0.9724973470109657
Training Flipped Recall score: 0.9313966291183197
Training Flipped F1 score: 0.9515033528012112
-----------------------------------

Dataset 2
-----------------------------------
Training Accuracy score: 0.21387193944543137
Training Precision score: 0.025629077353215284
Training Recall score: 0.04054054054054054
Training F1 score: 0.031404644080700414
-----------------------------------

-----------------------------------
Training Flipped Accuracy score: 0.7861280605545686
Training Flipped Precision score: 0.9743709226467847
Training Flipped Recall score: 0.706657654613045
Training Flipped F1 s

### Evaluate Model on Testing Data

In [133]:
for i in range(len(k_means)):
  print('Dataset '+ str(i+1))
  k = k_means[i]
  labels = np.uint(k.predict(test_data[i]))
  # flip the labels since the random assignment got them backwards
  labels_flipped = np.logical_not(labels).astype(int)
  print_analysis(labels, test_labels[i], 'Testing')
  print_analysis(labels_flipped, test_labels[i], 'Testing Flipped')



Dataset 1
-----------------------------------
Testing Accuracy score: 0.07785912148793035
Testing Precision score: 0.024644994718929704
Testing Recall score: 0.1721311475409836
Testing F1 score: 0.043116723129042196
-----------------------------------

-----------------------------------
Testing Flipped Accuracy score: 0.9221408785120696
Testing Flipped Precision score: 0.9753550052810703
Testing Flipped Recall score: 0.9350810081008101
Testing Flipped F1 score: 0.9547934976161756
-----------------------------------

Dataset 2
-----------------------------------
Testing Accuracy score: 0.19894964473277726
Testing Precision score: 0.02599758162031439
Testing Recall score: 0.041951219512195125
Testing F1 score: 0.03210153042179919
-----------------------------------

-----------------------------------
Testing Flipped Accuracy score: 0.8010503552672228
Testing Flipped Precision score: 0.9740024183796856
Testing Flipped Recall score: 0.7283001808318263
Testing Flipped F1 score: 0.83341955

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Prototype-Based Model: Gaussian Mixture

### Gaussian Mixture training and resulting centroids

In [134]:
gaussian = []
for i in range(len(train_data)):
  print('Dataset '+ str(i+1))
  data = pd.DataFrame(train_data[i], columns = train_df[i].columns[:-1])
  gm = GaussianMixture(n_components=2, random_state=1).fit(data)
  gaussian.append(gm)
  centroids = gm.means_
  print(pd.DataFrame(centroids, columns=data.columns))

Dataset 1
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.077743    -0.219195  ...         -0.111328       -0.134562
1  0.767573     2.164166  ...          1.099164        1.328558

[2 rows x 9 columns]
Dataset 2
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.167663    -0.200376  ...         -0.221606       -0.232353
1  0.526870     0.629668  ...          0.696383        0.730155

[2 rows x 9 columns]
Dataset 3
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.095495    -0.328203  ...         -0.187164       -0.173137
1  0.395642     1.359770  ...          0.775435        0.717318

[2 rows x 9 columns]
Dataset 4
   verified  geo_enabled  ...  favourites_count  statuses_count
0  0.540536     0.214335  ...          0.307285        0.379966
1 -0.211192    -0.083742  ...         -0.120059       -0.148456

[2 rows x 9 columns]


### Evaluate on Training Data

In [135]:
for i in range(len(gaussian)):
  print('Dataset '+ str(i+1))
  gm = gaussian[i]
  labels = np.uint(gm.predict(train_data[i]))
  # flip the labels since the random assignment got them backwards
  labels_flipped = np.logical_not(labels).astype(int)
  print_analysis(labels, train_labels[i], 'Training')
  print_analysis(labels_flipped, train_labels[i], 'Training Flipped')


Dataset 1
-----------------------------------
Training Accuracy score: 0.06980138019738306
Training Precision score: 0.0005305978068623983
Training Recall score: 0.004840010755579457
Training F1 score: 0.000956353106819329
-----------------------------------

-----------------------------------
Training Flipped Accuracy score: 0.930198619802617
Training Flipped Precision score: 0.9994694021931376
Training Flipped Recall score: 0.9236175429038409
Training Flipped F1 score: 0.9600475691593283
-----------------------------------

Dataset 2
-----------------------------------
Training Accuracy score: 0.28693905924152313
Training Precision score: 0.026250388319353838
Training Recall score: 0.05395913154533844
Training F1 score: 0.03531870428422153
-----------------------------------

-----------------------------------
Training Flipped Accuracy score: 0.7130609407584768
Training Flipped Precision score: 0.9737496116806461
Training Flipped Recall score: 0.6387162506367804
Training Flipped F1

### Evaluate on Testing Data

In [136]:
for i in range(len(gaussian)):
  print('Dataset '+ str(i+1))
  gm = gaussian[i]
  labels = np.uint(gm.predict(test_data[i]))
  # flip the labels since the random assignment got them backwards
  labels_flipped = np.logical_not(labels).astype(int)
  print_analysis(labels, test_labels[i], 'Testing')
  print_analysis(labels_flipped, test_labels[i], 'Testing Flipped')

Dataset 1
-----------------------------------
Testing Accuracy score: 0.06519588444796201
Testing Precision score: 0.0002347142354183781
Testing Recall score: 0.002145922746781116
Testing F1 score: 0.0004231460911879826
-----------------------------------

-----------------------------------
Testing Flipped Accuracy score: 0.934804115552038
Testing Flipped Precision score: 0.9997652857645816
Testing Flipped Recall score: 0.9284001743679163
Testing Flipped F1 score: 0.9627620500649827
-----------------------------------

Dataset 2
-----------------------------------
Testing Accuracy score: 0.2829780661105962
Testing Precision score: 0.02781136638452237
Testing Recall score: 0.06060606060606061
Testing F1 score: 0.03812681309573145
-----------------------------------

-----------------------------------
Testing Flipped Accuracy score: 0.7170219338894037
Testing Flipped Precision score: 0.9721886336154776
Testing Flipped Recall score: 0.648910411622276
Testing Flipped F1 score: 0.77831558

# Check for outliers

This appears to struggling with a single outlier so what if we remove it

In [137]:
for i in range(len(train_df)):
  df = train_df[i]
  print('train data')
  print(df[df['followers_count']>10000000])
  df = test_df[i]
  print('test data')
  print(df[df['followers_count']>10000000])

train data
    verified  geo_enabled  ...  statuses_count  bot
13      True         True  ...           11335    0

[1 rows x 10 columns]
test data
Empty DataFrame
Columns: [verified, geo_enabled, profile_use_background_image, default_profile, followers_count, friends_count, listed_count, favourites_count, statuses_count, bot]
Index: []
train data
    verified  geo_enabled  ...  statuses_count  bot
13      True         True  ...           11335    0

[1 rows x 10 columns]
test data
Empty DataFrame
Columns: [verified, geo_enabled, profile_use_background_image, default_profile, followers_count, friends_count, listed_count, favourites_count, statuses_count, bot]
Index: []
train data
    verified  geo_enabled  ...  statuses_count  bot
13      True         True  ...           11335    0

[1 rows x 10 columns]
test data
Empty DataFrame
Columns: [verified, geo_enabled, profile_use_background_image, default_profile, followers_count, friends_count, listed_count, favourites_count, statuses_count

In [138]:
train_data_new = []
train_labels_new = []
test_data_new = []
test_labels_new = []

for i in range(len(train_df)):
  train = train_df[i]
  test = test_df[i]
  train_new = train.drop(train.index[13])
  x_train_new = train_new.to_numpy()[:, :-1].astype(int)
  x_train_scaled_new = scaler.fit_transform(x_train_new)
  train_data_new.append(x_train_scaled_new)
  y_train = train_new.to_numpy()[:, -1].astype(int)
  train_labels_new.append(y_train)

  x_test = test.to_numpy()[:, :-1].astype(int)
  x_test_scaled = scaler.transform(x_test)
  test_data_new.append(x_test_scaled)
  y_test = test.to_numpy()[:, -1].astype(int)
  test_labels_new.append(y_test)

## Rerun on K-means

In [139]:
k_means = []
for i in range(len(train_data)):
  print('Dataset '+ str(i+1))
  data = pd.DataFrame(train_data_new[i], columns = train_df[i].columns[:-1])
  k = cluster.KMeans(n_clusters=2, max_iter=100, random_state=1)
  k.fit(data)
  k_means.append(k)
  centroids = k.cluster_centers_
  print(pd.DataFrame(centroids, columns=data.columns))

Dataset 1
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.089680    -0.237540  ...         -0.147124       -0.158356
1  0.634426     1.680429  ...          1.040799        1.120258

[2 rows x 9 columns]
Dataset 2
   verified  geo_enabled  ...  favourites_count  statuses_count
0  0.344568     0.923823  ...          0.514872        0.552266
1 -0.157998    -0.423610  ...         -0.236089       -0.253236

[2 rows x 9 columns]
Dataset 3
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.093630    -0.240179  ...         -0.148572       -0.149074
1  0.646829     1.659248  ...          1.026389        1.029859

[2 rows x 9 columns]
Dataset 4
   verified  geo_enabled  ...  favourites_count  statuses_count
0  0.204590     0.531314  ...          0.247752        0.298253
1 -0.194758    -0.505779  ...         -0.235845       -0.283919

[2 rows x 9 columns]


### Evaluate Models on Training Data

In [140]:
for i in range(len(k_means)):
  print('Dataset '+ str(i+1))
  k = k_means[i]
  labels = np.uint(k.predict(train_data_new[i]))
  # flip the labels since the random assignment got them backwards
  labels_flipped = np.logical_not(labels).astype(int)
  print_analysis(labels, train_labels_new[i], 'Training')
  print_analysis(labels_flipped, train_labels_new[i], 'Training Flipped')

Dataset 1
-----------------------------------
Training Accuracy score: 0.0831849213416444
Training Precision score: 0.027502652989034312
Training Recall score: 0.18633912522468543
Training F1 score: 0.04793095476612468
-----------------------------------

-----------------------------------
Training Flipped Accuracy score: 0.9168150786583555
Training Flipped Precision score: 0.9724973470109657
Training Flipped Recall score: 0.9313966291183197
Training Flipped F1 score: 0.9515033528012112
-----------------------------------

Dataset 2
-----------------------------------
Training Accuracy score: 0.7861115402440908
Training Precision score: 0.9743669411216405
Training Recall score: 0.7066246056782335
Training F1 score: 0.8191732514856658
-----------------------------------

-----------------------------------
Training Flipped Accuracy score: 0.21388845975590917
Training Flipped Precision score: 0.025633058878359483
Training Flipped Recall score: 0.04054054054054054
Training Flipped F1 sco

### Evaluate Models on Testing Data

In [141]:
for i in range(len(k_means)):
  print('Dataset '+ str(i+1))
  k = k_means[i]
  labels = np.uint(k.predict(test_data_new[i]))
  # flip the labels since the random assignment got them backwards
  labels_flipped = np.logical_not(labels).astype(int)
  print_analysis(labels, test_labels_new[i], 'Testing')
  print_analysis(labels_flipped, test_labels_new[i], 'Testing Flipped')



Dataset 1
-----------------------------------
Testing Accuracy score: 0.07785912148793035
Testing Precision score: 0.024644994718929704
Testing Recall score: 0.1721311475409836
Testing F1 score: 0.043116723129042196
-----------------------------------

-----------------------------------
Testing Flipped Accuracy score: 0.9221408785120696
Testing Flipped Precision score: 0.9753550052810703
Testing Flipped Recall score: 0.9350810081008101
Testing Flipped F1 score: 0.9547934976161756
-----------------------------------

Dataset 2
-----------------------------------
Testing Accuracy score: 0.8010503552672228
Testing Precision score: 0.9740024183796856
Testing Recall score: 0.7283001808318263
Testing F1 score: 0.8334195550957061
-----------------------------------

-----------------------------------
Testing Flipped Accuracy score: 0.19894964473277726
Testing Flipped Precision score: 0.02599758162031439
Testing Flipped Recall score: 0.041951219512195125
Testing Flipped F1 score: 0.032101530

## Rerun on Gaussian Mixture

In [142]:
gaussian = []
for i in range(len(train_data_new)):
  print('Dataset '+ str(i+1))
  data = pd.DataFrame(train_data_new[i], columns = train_df[i].columns[:-1])
  gm = GaussianMixture(n_components=2, random_state=1).fit(data)
  gaussian.append(gm)
  centroids = gm.means_
  print(pd.DataFrame(centroids, columns=data.columns))

Dataset 1
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.077744    -0.219142  ...         -0.111328       -0.134557
1  0.767793     2.164222  ...          1.099468        1.328869

[2 rows x 9 columns]
Dataset 2
   verified  geo_enabled  ...  favourites_count  statuses_count
0  0.526915     0.629585  ...          0.696382        0.730112
1 -0.167670    -0.200340  ...         -0.221596       -0.232329

[2 rows x 9 columns]
Dataset 3
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.095498    -0.328217  ...         -0.187172       -0.173144
1  0.395622     1.359705  ...          0.775396        0.717282

[2 rows x 9 columns]
Dataset 4
   verified  geo_enabled  ...  favourites_count  statuses_count
0 -0.210875    -0.083867  ...         -0.121490       -0.148903
1  0.539253     0.214467  ...          0.310675        0.380778

[2 rows x 9 columns]


### Evaluate Models on Training Data

In [143]:
for i in range(len(gaussian)):
  print('Dataset '+ str(i+1))
  gm = gaussian[i]
  labels = np.uint(gm.predict(train_data_new[i]))
  # flip the labels since the random assignment got them backwards
  labels_flipped = np.logical_not(labels).astype(int)
  print_analysis(labels, train_labels_new[i], 'Training')
  print_analysis(labels_flipped, train_labels_new[i], 'Training Flipped')


Dataset 1
-----------------------------------
Training Accuracy score: 0.06980310675769269
Training Precision score: 0.0005305978068623983
Training Recall score: 0.004841312533620226
Training F1 score: 0.0009563785133627332
-----------------------------------

-----------------------------------
Training Flipped Accuracy score: 0.9301968932423074
Training Flipped Precision score: 0.9994694021931376
Training Flipped Recall score: 0.9236175429038409
Training Flipped F1 score: 0.9600475691593283
-----------------------------------

Dataset 2
-----------------------------------
Training Accuracy score: 0.7130387764560482
Training Precision score: 0.9737455336336803
Training Recall score: 0.6386794375382108
Training F1 score: 0.7713986831579595
-----------------------------------

-----------------------------------
Training Flipped Accuracy score: 0.2869612235439518
Training Flipped Precision score: 0.026254466366319715
Training Flipped Recall score: 0.05395913154533844
Training Flipped F1

### Evaluate Models on Testing Data

In [144]:
for i in range(len(gaussian)):
  print('Dataset '+ str(i+1))
  gm = gaussian[i]
  labels = np.uint(gm.predict(test_data_new[i]))
  # flip the labels since the random assignment got them backwards
  labels_flipped = np.logical_not(labels).astype(int)
  print_analysis(labels, test_labels_new[i], 'Testing')
  print_analysis(labels_flipped, test_labels_new[i], 'Testing Flipped')

Dataset 1
-----------------------------------
Testing Accuracy score: 0.06519588444796201
Testing Precision score: 0.0002347142354183781
Testing Recall score: 0.002145922746781116
Testing F1 score: 0.0004231460911879826
-----------------------------------

-----------------------------------
Testing Flipped Accuracy score: 0.934804115552038
Testing Flipped Precision score: 0.9997652857645816
Testing Flipped Recall score: 0.9284001743679163
Testing Flipped F1 score: 0.9627620500649827
-----------------------------------

Dataset 2
-----------------------------------
Testing Accuracy score: 0.7170219338894037
Testing Precision score: 0.9721886336154776
Testing Recall score: 0.648910411622276
Testing F1 score: 0.7783155856727976
-----------------------------------

-----------------------------------
Testing Flipped Accuracy score: 0.2829780661105962
Testing Flipped Precision score: 0.02781136638452237
Testing Flipped Recall score: 0.06060606060606061
Testing Flipped F1 score: 0.038126813